In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

1. Cross-validation function

Write a function that receives data as input, does a 5-fold cross-validation and tests the following models (with
default hyper-parameters): **GaussianNB, LogisticRegression, RandomForestClassifier, ExtraTreesClassifier, and
XGBClassifier.**

The xgboost library is used for the XGBClassifier algorithm.

Other algorithms are located within the sklearn library.

The function has no return value but prints the table in the format shown below with the usage of f1_score as an
evaluation metric.

In [2]:
def cross_validate_models(data_train, data_test, features, submission_number):
    # Filter the training and testing data to include only the specified features plus 'Target'
    X_train = data_train[features]
    y_train = data_train['Target']
    X_test = data_test[features]

    # Define models
    models = {
        "GaussianNB": GaussianNB(),
        "LogisticRegression": LogisticRegression(max_iter=1000),
        "RandomForestClassifier": RandomForestClassifier(),
        "ExtraTreesClassifier": ExtraTreesClassifier(),
        "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    }

    # Define scoring method and folds
    skf = StratifiedKFold(n_splits=5)
    f1_scorer = make_scorer(f1_score)

    # Evaluate each model
    results_display = pd.DataFrame()
    for name, model in models.items():
        cv_scores = cross_val_score(model, X_train, y_train, cv=skf, scoring=f1_scorer)
        results_display[name] = np.append(cv_scores, cv_scores.mean())

    results_display.index = [f'Fold {i+1}' for i in range(5)] + ['Average']
    print(results_display.T)

    # Select the best model based on average F1 score
    best_model_name = results_display.mean(axis=0).idxmax()
    best_model = models[best_model_name]

    # Retrain the best model on the entire dataset
    best_model.fit(X_train, y_train)

    # Predict on the test dataset
    predictions = best_model.predict(X_test)

    # Create submission DataFrame
    submission = data_test.loc[:, ['Id']]
    submission['Target'] = predictions

    # Save submission to CSV
    filename = f"submission{submission_number}.csv"
    submission.to_csv(filename, index=False)


In [29]:
data_train = pd.read_csv("train_clean.csv")
data_test = pd.read_csv("test_clean.csv")
features = ['Close','High','Low','Open','Volume'] # Adj close is redundant with Close

cross_validate_models(data_train, data_test, features, 1)

                          Fold 1    Fold 2    Fold 3    Fold 4    Fold 5  \
GaussianNB              0.846106  0.877381  0.877484  0.877405  0.877482   
LogisticRegression      0.877473  0.877478  0.877478  0.877478  0.877478   
RandomForestClassifier  0.857616  0.864266  0.862423  0.862678  0.864373   
ExtraTreesClassifier    0.858159  0.866049  0.864665  0.866600  0.866736   
XGBClassifier           0.873122  0.876640  0.876431  0.877125  0.876989   

                         Average  
GaussianNB              0.871171  
LogisticRegression      0.877477  
RandomForestClassifier  0.862271  
ExtraTreesClassifier    0.864442  
XGBClassifier           0.876062  


**https://www.investopedia.com/top-7-technical-analysis-tools-4773275**

In [3]:
# Domain features

#1 - On-Balance Volume (OBV)

def calculate_obv(data):
    data = data.sort_values(by=['Symbol', 'Date'])
    data['OBV'] = 0  # Ensure OBV is initialized as an integer if that's the intended type
    for symbol in data['Symbol'].unique():
        symbol_data = data[data['Symbol'] == symbol].copy()
        price_changes = symbol_data['Close'].diff()
        volumes = symbol_data['Volume']
        obv_values = volumes.where(price_changes > 0, -volumes)
        obv_values.iloc[0] = 0  # Proper initialization
        symbol_data['OBV'] = obv_values.cumsum()
        symbol_data['OBV'] = symbol_data['OBV'].astype('int64')  # Explicit type casting
        data.loc[symbol_data.index, 'OBV'] = symbol_data['OBV']
        
    return data

#2 - Accumulation/Distribution line

def calculate_ad_line(data):
    # Calculate Money Flow Multiplier
    data['Money Flow Multiplier'] = ((data['Close'] - data['Low']) - (data['High'] - data['Close'])) / (data['High'] - data['Low'])
    # Calculate Money Flow Volume
    data['Money Flow Volume'] = data['Money Flow Multiplier'] * data['Volume']
    # Initialize AD Line in the DataFrame
    data['AD Line'] = 0
    
    # Calculate the AD Line without retaining intermediate columns
    data['AD Line'] = data['Money Flow Volume'].cumsum()

    # Drop intermediate columns to clean up DataFrame
    data.drop(['Money Flow Multiplier', 'Money Flow Volume'], axis=1, inplace=True)

    return data

#3 - Average Directional Index (ADX)

def calculate_adx(data, n=14):
    # Calculate the differences needed
    high_diff = data['High'].diff()
    low_diff = data['Low'].diff()

    # Calculate +DM and -DM
    data['+DM'] = np.where((high_diff > low_diff) & (high_diff > 0), high_diff, 0.0)
    data['-DM'] = np.where((low_diff > high_diff) & (low_diff > 0), low_diff, 0.0)

    # Calculate the True Range (TR)
    data['TR'] = np.maximum.reduce([data['High'] - data['Low'], 
                                    abs(data['High'] - data['Close'].shift()), 
                                    abs(data['Low'] - data['Close'].shift())])

    # Smooth the True Range and the Directional Movements
    data['TR14'] = data['TR'].rolling(window=n).mean()
    data['+DM14'] = data['+DM'].rolling(window=n).mean()
    data['-DM14'] = data['-DM'].rolling(window=n).mean()

    # Calculate Directional Indexes
    data['+DI14'] = 100 * data['+DM14'] / data['TR14']
    data['-DI14'] = 100 * data['-DM14'] / data['TR14']

    # Calculate the ADX
    data['DX'] = 100 * abs(data['+DI14'] - data['-DI14']) / (data['+DI14'] + data['-DI14'])
    data['ADX'] = data['DX'].rolling(window=n).mean()

    return data

#4 - Aroon Indicator (AROON)

def calculate_aroon(data, period=25):
    # Calculate Aroon Up and Aroon Down
    aroon_up = 100 * data['High'].rolling(window=period, min_periods=0).apply(
        lambda x: float(period - x.argmax()) / period, raw=True)
    aroon_down = 100 * data['Low'].rolling(window=period, min_periods=0).apply(
        lambda x: float(period - x.argmin()) / period, raw=True)

    # Append the indicators to the DataFrame
    data['Aroon Up'] = aroon_up
    data['Aroon Down'] = aroon_down

    return data

#5 - MACD

def calculate_macd(data):
    # Calculate the MACD and Signal Line indicators
    # Short term exponential moving average (EMA)
    ShortEMA = data.Close.ewm(span=12, adjust=False).mean() # 12-period EMA
    # Long term exponential moving average (EMA)
    LongEMA = data.Close.ewm(span=26, adjust=False).mean() # 26-period EMA
    # Calculate the MACD line
    data['MACD'] = ShortEMA - LongEMA
    # Calculate the signal line
    data['Signal_Line'] = data.MACD.ewm(span=9, adjust=False).mean()
    # Calculate the MACD Histogram
    data['MACD_Histogram'] = data['MACD'] - data['Signal_Line']
    
    return data

#6 - Relative Strength Index (RSI)

def calculate_rsi(data, periods=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    # Use the exponential moving average
    avg_gain = gain.ewm(span=periods, adjust=False).mean()
    avg_loss = loss.ewm(span=periods, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    data['RSI'] = rsi.fillna(0)  # Initializing RSI values to 0 before enough data
    return data


#7 - Stochastic Oscillator

def calculate_stochastic_oscillator(data, periods=14):
    low_min = data['Low'].rolling(window=periods, min_periods=1).min()
    high_max = data['High'].rolling(window=periods, min_periods=1).max()

    k = 100 * ((data['Close'] - low_min) / (high_max - low_min).replace(0, np.nan))
    data['Stochastic_%K'] = k
    data['Stochastic_%D'] = k.rolling(window=3, min_periods=1).mean()  # Moving average of %K
    return data







In [4]:
def add_historical_features(data):
    # Rolling Average Volume (20 days)
    data['Rolling_Avg_Vol_20'] = data['Volume'].rolling(window=20).mean()
    
    # Volatility
    data['Volatility_20'] = data['Close'].pct_change().rolling(window=20).std()
    
    # Relative Price Oscillator (RPO)
    short_ema = data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = data['Close'].ewm(span=26, adjust=False).mean()
    data['RPO'] = short_ema - long_ema
    
    # Cumulative Returns
    data['Cum_Return_1M'] = data['Close'].pct_change(20).cumsum()  # 20 trading days in a month
    data['Cum_Return_2M'] = data['Close'].pct_change(40).cumsum()  # 40 trading days in two months
    

    # Initialize columns
    data['Days_since_High'] = 0
    data['Days_since_Low'] = 0

    # Tracks the last occurrence index
    last_high = last_low = None

    # Iterate over DataFrame rows
    for index, row in data.iterrows():
        if last_high is None or row['Close'] >= data.loc[last_high, 'Close']:
            last_high = index
        if last_low is None or row['Close'] <= data.loc[last_low, 'Close']:
            last_low = index
        
        # Calculate days since last high and low
        data.at[index, 'Days_since_High'] = index - last_high
        data.at[index, 'Days_since_Low'] = index - last_low

    
    # Exponential Decay Moving Average
    data['EDMA'] = data['Close'].ewm(span=20, adjust=False).mean()
    
    # VWAP
    data['VWAP'] = (data['Volume'] * (data['High'] + data['Low'] + data['Close']) / 3).cumsum() / data['Volume'].cumsum()
    
    # Price Count
    data['Price_Up_Count'] = data['Close'].rolling(window=20).apply(lambda x: (np.diff(x) > 0).sum())
    data['Price_Down_Count'] = data['Close'].rolling(window=20).apply(lambda x: (np.diff(x) < 0).sum())
    
    # Gap Detection
    data['Gap'] = data['Open'] - data['Close'].shift(1)
    
    # Global Stock Rate Change (GSRC)

    # Calculate the total 'Close' for each day
    daily_totals = data.groupby('Date')['Close'].sum()

    # Calculate the day-to-day percentage change in total 'Close' values
    daily_change = daily_totals.pct_change().fillna(0)  # Replace NaN with 0 for the first day

    # Map these changes back to the original DataFrame based on the 'Date'
    data['Global_SRC'] = data['Date'].map(daily_change)
    return data


In [5]:
def add_features(data):
    data = calculate_obv(data)
    data = calculate_ad_line(data)
    data = calculate_adx(data)
    data = calculate_aroon(data)
    data = calculate_macd(data)
    data = calculate_rsi(data)
    data = calculate_stochastic_oscillator(data)

    data = add_historical_features(data)

    return data

In [37]:
data_train = pd.read_csv("train_clean.csv")
data_train = data_train.sort_values(by=['Symbol', 'Date'], ascending=[True, True])
data_test = pd.read_csv("test_clean.csv")
data_test = data_test.sort_values(by=['Symbol', 'Date'], ascending=[True, True])

features = ['Close', 'High', 'Low', 'Open', 'Volume', 'OBV', 'AD Line', 'ADX', 'Aroon Up', 'Aroon Down', 'MACD', 'RSI', 'Stochastic_%K', 'Stochastic_%D', 'Rolling_Avg_Vol_20', 'Volatility_20', 'RPO', 'Cum_Return_1M', 'Cum_Return_2M', 'EDMA', 'VWAP', 'Price_Up_Count', 'Price_Down_Count', 'Gap', 'Global_SRC']
data_train = add_features(data_train)
data_train = data_train.fillna(0)
data_test = add_features(data_test)
data_test = data_test.fillna(0)

print("training models")
cross_validate_models(data_train, data_test, features, 2)

training models
                          Fold 1    Fold 2    Fold 3    Fold 4    Fold 5  \
GaussianNB              0.851119  0.876911  0.877349  0.877257  0.877349   
LogisticRegression      0.853553  0.877339  0.877339  0.877345  0.877345   
RandomForestClassifier  0.862737  0.848261  0.807888  0.820517  0.877366   
ExtraTreesClassifier    0.872087  0.863588  0.837930  0.835468  0.877354   
XGBClassifier           0.702003  0.849078  0.729215  0.688516  0.877358   

                         Average  
GaussianNB              0.871997  
LogisticRegression      0.872584  
RandomForestClassifier  0.843354  
ExtraTreesClassifier    0.857285  
XGBClassifier           0.769234  


In [26]:
data_train = pd.read_csv("train_clean.csv")
data_train = data_train.sort_values(by=['Symbol', 'Date'], ascending=[True, True])
data_test = pd.read_csv("test_clean.csv")
data_test = data_test.sort_values(by=['Symbol', 'Date'], ascending=[True, True])

features = ['Close', 'High', 'Low', 'Open', 'Volume', 'OBV', 'AD Line', 'ADX', 'Aroon Up', 'Aroon Down', 'MACD', 'RSI', 'Stochastic_%K', 'Stochastic_%D', 'Rolling_Avg_Vol_20', 'Volatility_20', 'RPO', 'Cum_Return_1M', 'Cum_Return_2M', 'EDMA', 'VWAP', 'Price_Up_Count', 'Price_Down_Count', 'Gap', 'Global_SRC']
data_train = add_features(data_train)
data_train = data_train.fillna(0)
data_test = add_features(data_test)
data_test = data_test.fillna(0)

In [27]:
def feature_selection(train, initial_features):

    X = train[initial_features]
    y = train['Target']

    current_features = initial_features[:]
    while len(current_features) > 12:
        scores = []
        for feature in current_features:
            temp_features = [f for f in current_features if f != feature]
            X_train, X_test, y_train, y_test = train_test_split(X[temp_features], y, test_size=0.25, random_state=42)
            model = LinearSVC(dual=False, max_iter=5000)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test[temp_features])
            scores.append(f1_score(y_test, y_pred))

        # Determine which feature to remove
        best_score_index = np.argmax(scores)
        feature_to_remove = current_features[best_score_index]
        current_features.remove(feature_to_remove)
        print(f"Removed {feature_to_remove}, remaining features: {len(current_features)}")
    
    return current_features

In [28]:
def wrapper(data_train):
    features = ['OBV', 'AD Line', 'ADX', 'Aroon Up', 'Aroon Down', 'MACD', 'RSI', 
                'Stochastic_%K', 'Stochastic_%D', 'Rolling_Avg_Vol_20', 'RPO', 'EDMA', 
                'VWAP', 'Price_Up_Count', 'Price_Down_Count', 'Global_SRC']

    # Perform feature selection
    selected_features = feature_selection(data_train,  features)
    print("Selected features:", selected_features)

    return selected_features


selected_features = wrapper(data_train)

Removed OBV, remaining features: 15
Removed AD Line, remaining features: 14
Removed ADX, remaining features: 13
Removed Aroon Up, remaining features: 12
Selected features: ['Aroon Down', 'MACD', 'RSI', 'Stochastic_%K', 'Stochastic_%D', 'Rolling_Avg_Vol_20', 'RPO', 'EDMA', 'VWAP', 'Price_Up_Count', 'Price_Down_Count', 'Global_SRC']


In [73]:
cross_validate_models(data_train, data_test, selected_features, 3)

In [8]:
def plot_stock_trends(stock_data, symbol, features, ax=None):
    stock_specific_data = stock_data.loc[stock_data['Symbol'] == symbol].copy()
    stock_specific_data['Date'] = pd.to_datetime(stock_specific_data['Date'])
    
    # Create a new figure and axis object if not provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(14, 7))
    
    # Plot each feature on the y-axis
    for feature in features:
        ax.plot(stock_specific_data['Date'], stock_specific_data[feature], label=feature)
    
    # Formatting the plot
    ax.set_title(f'Stock Trends for {symbol}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Value')
    ax.legend()
    
    return ax

In [9]:
def identify_special_indices(data):

    indices = []
    for i in range(len(data)):
        if i < 5:  # Ensure there's enough data before the current index
            continue
        # Check if the current and next four entries have a target of 1
        if data['Target'].iloc[i:i+5].sum() == 5:
            # Check if there's at least one zero in the five days before the current day
            if (data['Target'].iloc[i-5:i] == 0).any():
                indices.extend(range(i, i+5))  # Add indices of the 5 consecutive days to the list

    return indices

# Example usage with your data:
selected_indices = identify_special_indices(data_train[data_train['Symbol'] == 'AAPL'])
print(selected_indices)


[106, 107, 108, 109, 110, 107, 108, 109, 110, 111, 108, 109, 110, 111, 112, 119, 120, 121, 122, 123, 120, 121, 122, 123, 124, 121, 122, 123, 124, 125, 122, 123, 124, 125, 126, 123, 124, 125, 126, 127, 301, 302, 303, 304, 305, 302, 303, 304, 305, 306, 303, 304, 305, 306, 307, 304, 305, 306, 307, 308, 315, 316, 317, 318, 319, 316, 317, 318, 319, 320, 317, 318, 319, 320, 321, 318, 319, 320, 321, 322, 319, 320, 321, 322, 323, 440, 441, 442, 443, 444, 441, 442, 443, 444, 445, 442, 443, 444, 445, 446, 443, 444, 445, 446, 447, 444, 445, 446, 447, 448, 453, 454, 455, 456, 457, 454, 455, 456, 457, 458, 455, 456, 457, 458, 459, 456, 457, 458, 459, 460, 457, 458, 459, 460, 461, 584, 585, 586, 587, 588, 585, 586, 587, 588, 589, 586, 587, 588, 589, 590, 587, 588, 589, 590, 591, 588, 589, 590, 591, 592, 787, 788, 789, 790, 791, 788, 789, 790, 791, 792, 789, 790, 791, 792, 793, 790, 791, 792, 793, 794, 791, 792, 793, 794, 795, 813, 814, 815, 816, 817, 814, 815, 816, 817, 818, 815, 816, 817, 818, 819,

In [13]:
import shap
from sklearn.ensemble import RandomForestClassifier

# Train the model
model = RandomForestClassifier(random_state=420)
model.fit(data_train[selected_features], data_train['Target'])

# Create SHAP explainer and values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(data_train[selected_features])

# Data isolation logic
indices = identify_special_indices(data_train)

# SHAP visualization
for i in indices:
    shap.force_plot(explainer.expected_value[1], shap_values[1][i,:], data_train.iloc[i,:])

# Plotting
plot_stock_trends(data_train, 'AAPL', ['Close', 'Volume', 'ADX'])


In [30]:
import wittgenstein as lw
stock_data = data_train[data_train['Symbol'] == 'AAPL']

# Split the data
train, test = train_test_split(stock_data, test_size=0.33, random_state=420)

ripper = lw.RIPPER(random_state=42)
ripper.fit(train.drop('Target', axis=1), train['Target'])

# Predict and calculate F1 score
preds = ripper.predict(test.drop('Target', axis=1))
print(f"F1 Score: {f1_score(test['Target'], preds)}")

# Print the rules
print(ripper.ruleset_)

F1 Score: 0.9141965678627145
[[VWAP=15.1-15.88] V [Cum_Return_2M=109.62-116.99] V [Id=44369.2-44621.4] V [EDMA=<12.14] V [VWAP=>19.64^TR14=>1.06] V [EDMA=15.47-18.89^VWAP=14.7-15.1^ADLine=13693026082.55-15561189180.52] V [ADLine=15561189180.52-16577130747.53^-DM=<0.082^Open=15.14-18.61] V [Id=44116.4-44369.2^OBV=7233400160.0-8976385520.0] V [EDMA=12.14-15.47^VWAP=11.12-13.06] V [TR14=0.75-1.06^OBV=12071871760.0-12482838640.0] V [Rolling_Avg_Vol_20=227451240.0-298296040.0^AdjClose=25.66-29.11] V [VWAP=16.98-17.49^Cum_Return_1M=45.62-48.03^OBV=8976385520.0-9993894400.0] V [Rolling_Avg_Vol_20=382010580.0-479008964.0^Signal_Line=-0.05-0.05] V [Rolling_Avg_Vol_20=<103363488.0^+DM14=>0.2] V [OBV=>14534347600.0^High=18.8-21.77] V [AdjClose=22.45-25.66^ADX=>50.68] V [AdjClose=22.45-25.66^ADX=39.89-50.68] V [MACD=-0.056-0.047^AroonDown=56.0-72.0] V [Rolling_Avg_Vol_20=382010580.0-479008964.0^Volatility_20=0.014-0.016] V [-DM14=0.078-0.089^Close=15.17-18.61]]


In [31]:
print("Rules:")
for rule in ripper.ruleset_:
    print(rule)

Rules:
[VWAP=15.1-15.88]
[Cum_Return_2M=109.62-116.99]
[Id=44369.2-44621.4]
[EDMA=<12.14]
[VWAP=>19.64^TR14=>1.06]
[EDMA=15.47-18.89^VWAP=14.7-15.1^ADLine=13693026082.55-15561189180.52]
[ADLine=15561189180.52-16577130747.53^-DM=<0.082^Open=15.14-18.61]
[Id=44116.4-44369.2^OBV=7233400160.0-8976385520.0]
[EDMA=12.14-15.47^VWAP=11.12-13.06]
[TR14=0.75-1.06^OBV=12071871760.0-12482838640.0]
[Rolling_Avg_Vol_20=227451240.0-298296040.0^AdjClose=25.66-29.11]
[VWAP=16.98-17.49^Cum_Return_1M=45.62-48.03^OBV=8976385520.0-9993894400.0]
[Rolling_Avg_Vol_20=382010580.0-479008964.0^Signal_Line=-0.05-0.05]
[Rolling_Avg_Vol_20=<103363488.0^+DM14=>0.2]
[OBV=>14534347600.0^High=18.8-21.77]
[AdjClose=22.45-25.66^ADX=>50.68]
[AdjClose=22.45-25.66^ADX=39.89-50.68]
[MACD=-0.056-0.047^AroonDown=56.0-72.0]
[Rolling_Avg_Vol_20=382010580.0-479008964.0^Volatility_20=0.014-0.016]
[-DM14=0.078-0.089^Close=15.17-18.61]


In [32]:
param_grid = {
    'k': [1, 2, 3, 5],
    'prune_size': [0.1, 0.2, 0.33],
    'dl_allowance': [0.1, 0.5, 1]
}

# Wrapper to use RIPPER with GridSearchCV
def RIPPER_GridSearch(X, y, param_grid, cv=3):
    best_score = 0
    best_params = {}
    for k in param_grid['k']:
        for prune_size in param_grid['prune_size']:
            for dl_allowance in param_grid['dl_allowance']:
                model = lw.RIPPER(k=k, prune_size=prune_size, dl_allowance=dl_allowance)
                model.fit(X, y)
                score = f1_score(y, model.predict(X))
                if score > best_score:
                    best_score = score
                    best_params = {'k': k, 'prune_size': prune_size, 'dl_allowance': dl_allowance}
    return best_score, best_params

best_score, best_params = RIPPER_GridSearch(train.drop('Target', axis=1), train['Target'], param_grid)
print("Best Score:", best_score)
print("Best Params:", best_params)


ripper = lw.RIPPER(max_rules=3, max_rule_conds=2, **best_params)
ripper.fit(train.drop('Target', axis=1), train['Target'])
print(ripper.ruleset_)


Best Score: 0.9333838001514005
Best Params: {'k': 3, 'prune_size': 0.33, 'dl_allowance': 0.5}
[[Id=44369.2-44621.4] V [VWAP=15.1-15.88] V [Cum_Return_2M=109.62-116.99]]
